In [1]:
using CausalForest
using RCall
using StatsBase
using BenchmarkTools

┌ Info: Precompiling CausalForest [31bdf679-24a7-499a-9be9-b214c66174ea]
└ @ Base loading.jl:1423


In [2]:
indices,X,T,Y = load_data("causal")
reval("set.seed(1)")
Xtest_1 = rcopy(R"matrix(runif(10000),nrow=1000)")
true_effect_1 = Xtest_1[:, 1].>0.5;

# Critère exact : centré honnête

### Subsampling

In [3]:
errors_7 = zeros(100)

cf = @btime build_forest_opti(true, false, true, Y, T, X, true, 10, 500, 0.5, 0.5, -1, 5, 10, 2000)
pred = @btime apply_forest(cf, Xtest_1)
errors_7[1] = rmsd(float(true_effect_1), pred)
for i in 2:100
    @rput i
    reval("set.seed(i)")
    Xtest = rcopy(R"matrix(runif(10000),nrow=1000)")
    pred = apply_forest(cf, Xtest)
    true_effect = Xtest[:, 1].>0.5
    errors_7[i] = rmsd(float(true_effect), pred)
end
print(mean(errors_7))
print(var(errors_7))

  68.455 s (46791895 allocations: 5.96 GiB)
  79.676 s (3111009490 allocations: 46.60 GiB)
0.50118425547573651.0344825575204384e-6

### Bootstrap

In [4]:
errors_8 = zeros(100)

cf = @btime build_forest_opti(true, true, true, Y, T, X, true, 10, 500, 0.5, 0.5, -1, 5, 10, 2000)
pred = @btime apply_forest(cf, Xtest_1)
errors_8[1] = rmsd(float(true_effect_1), pred)
for i in 2:100
    @rput i
    reval("set.seed(i)")
    Xtest = rcopy(R"matrix(runif(10000),nrow=1000)")
    pred = apply_forest(cf, Xtest)
    true_effect = Xtest[:, 1].>0.5
    errors_8[i] = rmsd(float(true_effect), pred)
end
print(mean(errors_8))
print(var(errors_8))

  1028.749 s (47438683 allocations: 6.16 GiB)
  197.994 s (6108459490 allocations: 91.26 GiB)
0.5012260451773011.071249021068289e-6

# Critère alternatif (différence non centrée) honnête

### Subsampling

In [3]:
errors_11 = zeros(100)

cf = @btime build_forest_1_opti(false, false, true, Y, T, X, true, 10, 500, 0.5, 0.5, -1, 5, 10, 2000)
pred = @btime apply_forest_1(cf, Xtest_1)
errors_11[1] = rmsd(float(true_effect_1), pred)
for i in 2:100
    @rput i
    reval("set.seed(i)")
    Xtest = rcopy(R"matrix(runif(10000),nrow=1000)")
    pred = apply_forest_1(cf, Xtest)
    true_effect = Xtest[:, 1].>0.5
    errors_11[i] = rmsd(float(true_effect), pred)
end
print(mean(errors_11))
print(var(errors_11))

  2.249 s (951866 allocations: 357.26 MiB)
  5.137 s (186000598 allocations: 3.01 GiB)
0.068403557731431613.908746180886685e-5

### Bootstrap

In [4]:
errors_12 = zeros(100)

cf = @btime build_forest_1_opti(false, true, true, Y, T, X, true, 10, 500, 0.5, 0.5, -1, 5, 10, 2000)
pred = @btime apply_forest_1(cf, Xtest_1)
errors_12[1] = rmsd(float(true_effect_1), pred)
for i in 2:100
    @rput i
    reval("set.seed(i)")
    Xtest = rcopy(R"matrix(runif(10000),nrow=1000)")
    pred = apply_forest_1(cf, Xtest)
    true_effect = Xtest[:, 1].>0.5
    errors_12[i] = rmsd(float(true_effect), pred)
end
print(mean(errors_12))
print(var(errors_12))

  6.099 s (1866588 allocations: 585.13 MiB)
  5.355 s (194938514 allocations: 3.14 GiB)
0.072098917751472173.752496659951461e-5